In [ ]:
from pathlib import Path

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB and Path.cwd().name == "content":
    if not Path("quantum-jam-chirimbolo").exists():
        !git clone https://github.com/segusantos/quantum-jam-chirimbolo.git
        !pip install quantum-jam-chirimbolo/
    %cd quantum-jam-chirimbolo


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import HTML, display
import ipywidgets as widgets

from src import (
    BB84Protocol,
    BB84Parameters,
    NoiseChannel,
    CascadeErrorCorrector,
    PrivacyAmplifier,
)

plt.style.use("seaborn-v0_8-darkgrid")
pd.options.display.max_rows = 200
pd.options.display.max_columns = 20


# Protocolo BB84 con Qiskit

Exploramos un flujo completo de distribución de claves cuánticas BB84, desde la preparación de qubits hasta la corrección de errores y la amplificación de privacidad. Cada sección del cuaderno incorpora controles interactivos para experimentar con los parámetros del protocolo y visualizar métricas clave.


## Arquitectura del prototipo

- `BB84Protocol` orquesta la generación de estados, el ataque de Eve y las medidas de Bob empleando `AerSimulator`.
- `NoiseChannel` define canales de ruido configurables (depolarizante, bit/phase flip, amortiguamiento, etc.).
- `CascadeErrorCorrector` aplica corrección de errores clásica sobre la clave cribada.
- `PrivacyAmplifier` reduce la clave para absorber la fuga de información estimada.

La celdas siguientes usan estas primitivas para construir una narrativa interactiva en cuatro etapas.


In [ ]:
NOISE_PRESETS = {
    "Sin ruido": {"name": "none", "param": None, "label": "", "max": 0.0},
    "Depolarizante": {"name": "depolarizing", "param": "p", "label": "p", "max": 0.3},
    "Bit flip": {"name": "bit_flip", "param": "p", "label": "p", "max": 0.3},
    "Phase flip": {"name": "phase_flip", "param": "p", "label": "p", "max": 0.3},
    "Amortiguamiento de amplitud": {"name": "amplitude_damping", "param": "gamma", "label": "gamma", "max": 0.3},
    "Damping de fase": {"name": "phase_damping", "param": "lambda", "label": "lambda", "max": 0.3},
}


def clamp(value, lower=0.0, upper=1.0):
    return max(lower, min(upper, value))


def normalize_seed(raw):
    if raw is None:
        return None
    text = str(raw).strip()
    if not text:
        return None
    try:
        return int(text)
    except ValueError:
        return None


def build_noise_channel(option, value, readout0, readout1):
    preset = NOISE_PRESETS[option]
    params = {}
    if preset["param"]:
        params[preset["param"]] = clamp(value)
    if readout0 > 0:
        params["p0to1"] = clamp(readout0)
    if readout1 > 0:
        params["p1to0"] = clamp(readout1)
    return NoiseChannel(preset["name"], params)


def run_bb84(num_bits, seed_value, eve, eve_prob, noise_option, noise_value, readout0, readout1):
    seed = normalize_seed(seed_value)
    params = BB84Parameters(
        num_bits=num_bits,
        seed=seed,
        eve_present=eve,
        eve_intercept_prob=clamp(eve_prob),
    )
    params.noise = build_noise_channel(noise_option, noise_value, readout0, readout1)
    protocol = BB84Protocol(params)
    return protocol.run()


def style_result_table(result):
    df = result.to_dataframe()

    def highlight(row):
        base = "#ffffff"
        if row["Sifted"] == "Si":
            base = "#edf7ed"
        if row["Coincide?"] == "❌":
            if row["Causa"] == "Eve":
                base = "#fdecea"
            elif row["Causa"] == "Ruido":
                base = "#fff4e5"
            else:
                base = "#ede7f6"
        return [f"background-color: {base}"] * len(row)

    styled = df.style.apply(highlight, axis=1).set_properties(**{"text-align": "center"})
    try:
        styled = styled.hide(axis="index")
    except AttributeError:
        styled = styled.hide_index()
    return styled


def lerp_color(color_a, color_b, t):
    return tuple(int((1 - t) * a + t * b) for a, b in zip(color_a, color_b))


def color_to_hex(color):
    return f"#{color[0]:02x}{color[1]:02x}{color[2]:02x}"


def detection_badge(prob, label):
    value = clamp(prob)
    color = lerp_color((235, 87, 87), (67, 160, 71), value)
    return (
        f"<span style='display:inline-block;margin-right:8px;padding:6px 12px;border-radius:8px;"
        f"background:{color_to_hex(color)};color:#102a43;font-weight:600;'>{label}: {value:.6f}</span>"
    )


def key_summary(result):
    lines = [
        "RESULTADO BB84",
        "------------------------------------------",
        f"Bases iguales : {result.equal_bases()}",
        f"QBER          : {result.qber:.4f}",
        f"Clave Alice   : {result.sifted_alice_bits or '-'}",
        f"Clave Bob     : {result.sifted_bob_bits or '-'}",
    ]
    return HTML("<pre>" + "\n".join(lines) + "</pre>")


def detection_summary(result, detection_bits):
    sample = min(detection_bits, result.sifted_key_length())
    if sample == 0:
        return HTML("<p><strong>No hay suficientes bits cribados para el muestreo.</strong></p>")
    empirical = result.detection_probability(sample)
    ideal = 1.0 - (0.75 ** sample)
    mismatches = sum(1 for i in range(sample) if result.sifted_alice_bits[i] != result.sifted_bob_bits[i])
    badges = detection_badge(empirical, "P(det) empirica")
    if result.params.eve_present:
        badges += detection_badge(ideal, "P(det) ideal")
    body = (
        "<div style='margin-top:6px;'>"
        + badges
        + f"<div style='margin-top:4px;'>Bits muestreados: {sample} | discrepancias: {mismatches}</div>"
        + "</div>"
    )
    return HTML(body)


def split_after_detection(result, detection_bits):
    sample = min(detection_bits, result.sifted_key_length())
    return (
        result.sifted_alice_bits[sample:],
        result.sifted_bob_bits[sample:],
        result.sifted_alice_bits[:sample],
        result.sifted_bob_bits[:sample],
    )


def sweep_noise(num_bits, seed_value, eve, eve_prob, noise_option, values, readout0, readout1, detection_bits):
    data = []
    for value in values:
        result = run_bb84(num_bits, seed_value, eve, eve_prob, noise_option, value, readout0, readout1)
        sample = min(detection_bits, result.sifted_key_length())
        data.append({"value": value, "qber": result.qber, "p_detect": result.detection_probability(sample)})
    return data


def render_noise_curves(data, noise_label):
    if not data:
        return None
    values = [item["value"] for item in data]
    qber = [item["qber"] for item in data]
    p_detect = [item["p_detect"] for item in data]
    fig, ax = plt.subplots(figsize=(6, 3.5))
    ax.plot(values, qber, marker="o", color="#1f77b4", label="QBER")
    ax.set_xlabel(f"Parametro de ruido ({noise_label})")
    ax.set_ylabel("QBER")
    ax.grid(alpha=0.25)
    twin = ax.twinx()
    twin.plot(values, p_detect, marker="s", color="#2e7d32", label="P(det)")
    twin.set_ylabel("P(det)")
    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = twin.get_legend_handles_labels()
    ax.legend(lines + lines2, labels + labels2, loc="upper left")
    plt.tight_layout()
    return fig


def format_key_preview(key, limit=64):
    if not key:
        return "-"
    if len(key) <= limit:
        return key
    head = max(limit // 2, 1)
    tail = max(limit - head - 3, 0)
    if tail <= 0:
        return key[:limit]
    return key[:head] + "..." + key[-tail:]


def configure_noise_slider(dropdown, slider):
    def update(_=None):
        preset = NOISE_PRESETS[dropdown.value]
        slider.description = preset["label"] or "valor"
        slider.max = preset["max"]
        slider.step = max(preset["max"] / 50.0, 0.01) if preset["max"] > 0 else 0.01
        slider.disabled = preset["param"] is None
        if slider.max == 0:
            slider.value = 0.0
        elif slider.value > slider.max:
            slider.value = slider.max
    dropdown.observe(update, names="value")
    update()


def bind_controls(controls, callback, output):
    def handler(_=None):
        kwargs = {name: control.value for name, control in controls.items()}
        with output:
            output.clear_output(wait=True)
            callback(**kwargs)
    for control in controls.values():
        control.observe(handler, names="value")
    handler()
    return handler


## Parte 1 — Ejecución básica de BB84

Ajusta el número de bits generados por Alice y observa cómo se producen las coincidencias de bases y la clave cribada compartida.


In [ ]:
part1_slider = widgets.IntSlider(value=12, min=4, max=64, step=1, description="Bits (N)")
part1_output = widgets.Output()

def render_part1(num_bits):
    result = run_bb84(
        num_bits=num_bits,
        seed_value="",
        eve=False,
        eve_prob=0.0,
        noise_option="Sin ruido",
        noise_value=0.0,
        readout0=0.0,
        readout1=0.0,
    )
    display(key_summary(result))
    display(style_result_table(result))

bind_controls({"num_bits": part1_slider}, render_part1, part1_output)
display(widgets.VBox([part1_slider, part1_output]))


## Parte 2 — Ataque de Eve y muestreo de detección

Se incorpora una semilla configurable para reproducir experimentos, además de un ataque de interceptación-resend por parte de Eve. Ajusta la probabilidad de interceptación y la cantidad de bits `m` usados para detección.


In [ ]:
part2_num_bits = widgets.IntSlider(value=24, min=8, max=128, step=2, description="Bits (N)")
part2_seed = widgets.Text(value="", description="Semilla", placeholder="vacio = aleatorio")
part2_eve = widgets.Checkbox(value=True, description="Eve presente")
part2_eve_prob = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description="P(intercept)")
part2_detection = widgets.IntSlider(value=12, min=2, max=64, step=1, description="m (deteccion)")
part2_output = widgets.Output()


def _toggle_eve_slider(change):
    part2_eve_prob.disabled = not change["new"]


part2_eve.observe(_toggle_eve_slider, names="value")
part2_eve_prob.disabled = not part2_eve.value


def render_part2(num_bits, seed_value, eve, eve_prob, detection_bits):
    result = run_bb84(
        num_bits=num_bits,
        seed_value=seed_value,
        eve=eve,
        eve_prob=eve_prob,
        noise_option="Sin ruido",
        noise_value=0.0,
        readout0=0.0,
        readout1=0.0,
    )
    display(key_summary(result))
    display(style_result_table(result))
    display(detection_summary(result, detection_bits))


controls_part2 = {
    "num_bits": part2_num_bits,
    "seed_value": part2_seed,
    "eve": part2_eve,
    "eve_prob": part2_eve_prob,
    "detection_bits": part2_detection,
}

bind_controls(controls_part2, render_part2, part2_output)
display(
    widgets.VBox(
        [
            widgets.HBox([part2_num_bits, part2_detection]),
            widgets.HBox([part2_seed, part2_eve, part2_eve_prob]),
            part2_output,
        ]
    )
)


## Parte 3 — Explorando el impacto del ruido en BB84

Esta sección extiende la anterior e introduce canales de ruido configurables mediante `AerSimulator`. Ajusta el tipo de ruido y observa cómo varían el QBER y la probabilidad de detección.


In [ ]:
part3_num_bits = widgets.IntSlider(value=48, min=16, max=256, step=4, description="Bits (N)")
part3_seed = widgets.Text(value="", description="Semilla", placeholder="vacio = aleatorio")
part3_eve = widgets.Checkbox(value=True, description="Eve presente")
part3_eve_prob = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.05, description="P(intercept)")
part3_detection = widgets.IntSlider(value=16, min=4, max=96, step=2, description="m (deteccion)")
part3_noise = widgets.Dropdown(options=list(NOISE_PRESETS.keys()), value="Depolarizante", description="Ruido")
part3_noise_value = widgets.FloatSlider(value=0.1, min=0.0, max=0.3, step=0.01, description="p")
part3_readout0 = widgets.FloatSlider(value=0.0, min=0.0, max=0.2, step=0.01, description="p0->1")
part3_readout1 = widgets.FloatSlider(value=0.0, min=0.0, max=0.2, step=0.01, description="p1->0")
part3_points = widgets.IntSlider(value=7, min=3, max=15, step=1, description="barrido")
part3_output = widgets.Output()


configure_noise_slider(part3_noise, part3_noise_value)


def _toggle_part3_eve(change):
    part3_eve_prob.disabled = not change["new"]


part3_eve.observe(_toggle_part3_eve, names="value")
part3_eve_prob.disabled = not part3_eve.value


def render_part3(
    num_bits,
    seed_value,
    eve,
    eve_prob,
    detection_bits,
    noise_option,
    noise_value,
    readout0,
    readout1,
    sweep_points,
):
    result = run_bb84(
        num_bits=num_bits,
        seed_value=seed_value,
        eve=eve,
        eve_prob=eve_prob,
        noise_option=noise_option,
        noise_value=noise_value,
        readout0=readout0,
        readout1=readout1,
    )
    display(key_summary(result))
    sample = min(detection_bits, result.sifted_key_length())
    info = (
        f"<p>Bits cribados: <strong>{result.sifted_key_length()}</strong> | "
        f"QBER: <strong>{result.qber:.4f}</strong> | Muestreo m: {sample}</p>"
    )
    display(HTML(info))
    display(style_result_table(result))
    display(detection_summary(result, detection_bits))

    preset = NOISE_PRESETS[noise_option]
    data = []
    if preset["param"] and sweep_points > 1:
        upper = preset["max"] if preset["max"] > 0 else max(noise_value, 0.1)
        upper = max(upper, noise_value if noise_value > 0 else 0.1)
        values = np.linspace(0.0, upper, sweep_points)
        values = np.unique(np.append(values, noise_value))
        data = sweep_noise(
            num_bits,
            seed_value,
            eve,
            eve_prob,
            noise_option,
            values,
            readout0,
            readout1,
            detection_bits,
        )
    fig = render_noise_curves(data, preset["label"] or "valor")
    if fig is not None:
        display(fig)
        plt.close(fig)


controls_part3 = {
    "num_bits": part3_num_bits,
    "seed_value": part3_seed,
    "eve": part3_eve,
    "eve_prob": part3_eve_prob,
    "detection_bits": part3_detection,
    "noise_option": part3_noise,
    "noise_value": part3_noise_value,
    "readout0": part3_readout0,
    "readout1": part3_readout1,
    "sweep_points": part3_points,
}

bind_controls(controls_part3, render_part3, part3_output)
display(
    widgets.VBox(
        [
            widgets.HBox([part3_num_bits, part3_detection, part3_points]),
            widgets.HBox([part3_seed, part3_eve, part3_eve_prob]),
            widgets.HBox([part3_noise, part3_noise_value]),
            widgets.HBox([part3_readout0, part3_readout1]),
            part3_output,
        ]
    )
)


## Parte 4 — Corrección de errores y amplificación de privacidad

Aplicamos el algoritmo Cascade para corregir discrepancias residuales y posteriormente ejecutamos privacidad amplificada con SHAKE-256 truncado.


In [ ]:
part4_num_bits = widgets.IntSlider(value=64, min=16, max=256, step=4, description="Bits (N)")
part4_seed = widgets.Text(value="", description="Semilla", placeholder="vacio = aleatorio")
part4_eve = widgets.Checkbox(value=True, description="Eve presente")
part4_eve_prob = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, step=0.05, description="P(intercept)")
part4_detection = widgets.IntSlider(value=20, min=4, max=128, step=2, description="m (deteccion)")
part4_noise = widgets.Dropdown(options=list(NOISE_PRESETS.keys()), value="Depolarizante", description="Ruido")
part4_noise_value = widgets.FloatSlider(value=0.05, min=0.0, max=0.3, step=0.01, description="p")
part4_readout0 = widgets.FloatSlider(value=0.0, min=0.0, max=0.2, step=0.01, description="p0->1")
part4_readout1 = widgets.FloatSlider(value=0.0, min=0.0, max=0.2, step=0.01, description="p1->0")
part4_block = widgets.IntSlider(value=4, min=2, max=16, step=1, description="bloque")
part4_rounds = widgets.IntSlider(value=3, min=1, max=5, step=1, description="rondas")
part4_security = widgets.IntSlider(value=40, min=10, max=80, step=5, description="seguridad")
part4_target = widgets.IntSlider(value=0, min=0, max=128, step=4, description="target (0 auto)")
part4_output = widgets.Output()


configure_noise_slider(part4_noise, part4_noise_value)


def _toggle_part4_eve(change):
    part4_eve_prob.disabled = not change["new"]


part4_eve.observe(_toggle_part4_eve, names="value")
part4_eve_prob.disabled = not part4_eve.value


def render_part4(
    num_bits,
    seed_value,
    eve,
    eve_prob,
    detection_bits,
    noise_option,
    noise_value,
    readout0,
    readout1,
    bloque,
    rondas,
    seguridad,
    target_bits,
):
    result = run_bb84(
        num_bits=num_bits,
        seed_value=seed_value,
        eve=eve,
        eve_prob=eve_prob,
        noise_option=noise_option,
        noise_value=noise_value,
        readout0=readout0,
        readout1=readout1,
    )
    display(key_summary(result))
    display(style_result_table(result))
    display(detection_summary(result, detection_bits))

    alice_rest, bob_rest, alice_sample, bob_sample = split_after_detection(result, detection_bits)
    sample = len(alice_sample)
    sample_mismatches = sum(1 for a, b in zip(alice_sample, bob_sample) if a != b)
    detection_lines = [
        "MUESTREO PARA DETECCION",
        "------------------------------------------",
        f"m                 : {sample}",
        f"Discrepancias     : {sample_mismatches}",
        f"Alice (m)         : {format_key_preview(alice_sample, 48)}",
        f"Bob   (m)         : {format_key_preview(bob_sample, 48)}",
    ]
    display(HTML("<pre>" + "\n".join(detection_lines) + "</pre>"))

    if not alice_rest:
        display(HTML("<p><strong>No quedan bits tras el muestreo. Disminuye m o incrementa N.</strong></p>"))
        return

    corrector = CascadeErrorCorrector(block_size=bloque, rounds=rondas)
    cascade = corrector.correct(alice_rest, bob_rest)
    cascade_lines = [
        "CASCADE (correccion)",
        "------------------------------------------",
        f"Longitud entrada  : {len(alice_rest)}",
        f"Bloque inicial    : {bloque}",
        f"Rondas            : {rondas}",
        f"Correcciones      : {len(cascade.corrections)}",
        f"Bits fugados      : {cascade.leakage_bits}",
        f"Errores residuales: {cascade.residual_errors}",
        f"Clave corregida   : {format_key_preview(cascade.corrected_key, 64)}",
    ]
    display(HTML("<pre>" + "\n".join(cascade_lines) + "</pre>"))

    amplifier = PrivacyAmplifier(security_parameter=seguridad)
    target = None if target_bits <= 0 else target_bits
    privacy = amplifier.apply(cascade.corrected_key, cascade.leakage_bits, target_length=target)
    privacy_lines = [
        "PRIVACY AMPLIFICATION",
        "------------------------------------------",
        f"Hash              : {privacy.hash_function}",
        f"Seguridad (bits)  : {seguridad}",
        f"Target solicitado : {target if target is not None else 'auto'}",
        f"Longitud final    : {privacy.target_length}",
        f"Bits descartados  : {privacy.discarded_bits}",
        f"Clave final       : {format_key_preview(privacy.final_key, 64)}",
    ]
    display(HTML("<pre>" + "\n".join(privacy_lines) + "</pre>"))

    if cascade.residual_errors > 0:
        display(HTML("<p style='color:#c62828;font-weight:600;'>Persisten errores tras Cascade; aumenta rondas o m.</p>"))


controls_part4 = {
    "num_bits": part4_num_bits,
    "seed_value": part4_seed,
    "eve": part4_eve,
    "eve_prob": part4_eve_prob,
    "detection_bits": part4_detection,
    "noise_option": part4_noise,
    "noise_value": part4_noise_value,
    "readout0": part4_readout0,
    "readout1": part4_readout1,
    "bloque": part4_block,
    "rondas": part4_rounds,
    "seguridad": part4_security,
    "target_bits": part4_target,
}

bind_controls(controls_part4, render_part4, part4_output)
display(
    widgets.VBox(
        [
            widgets.HBox([part4_num_bits, part4_detection]),
            widgets.HBox([part4_seed, part4_eve, part4_eve_prob]),
            widgets.HBox([part4_noise, part4_noise_value]),
            widgets.HBox([part4_readout0, part4_readout1]),
            widgets.HBox([part4_block, part4_rounds, part4_security, part4_target]),
            part4_output,
        ]
    )
)


## Conclusiones y próximos pasos

- Explora valores más altos de `N` para obtener estadísticas más estables.
- Ajusta `m`, las rondas de Cascade y el parámetro de seguridad para equilibrar detección y longitud final.
- Conecta el cuaderno a un backend real de IBM Quantum para contrastar con hardware físico.
